In [1]:
import ipywidgets as widgets
from IPython.display import display, Video, HTML, clear_output
from ipywidgets import Button, HBox, Layout

print(widgets.__version__)

# print([method_name for method_name in dir(object)
#                   if callable(getattr(object, method_name))])

speed = widgets.FloatSlider(
    min=0.1,
    max=2,
    step=0.1,
    description='Playbackspeed:',
    value=1
)

display(speed)

7.7.2


FloatSlider(value=1.0, description='Playbackspeed:', max=2.0, min=0.1)

In [ ]:
timestamp = 2.0
window_size = 2


def display_html(timestamp, window_size):
    print(timestamp, window_size)
    display(HTML(f'''
        <body>
            <video id="window" width="500px"; class="center" src="Timer2.mp4"></video><br>
            <button onclick="play()">Play</button>
            <button onclick="pause()">Pause</button>
            <script type="text/javascript">
                var timestamp = {timestamp}
                var window_size = {window_size}

                function play(){{
                    var video= document.getElementById("window");
                    if (video.currentTime < timestamp || video.currentTime >= timestamp + window_size){{
                        video.currentTime = timestamp;
                    }}
                    video.play()
                    setInterval(function(){{
                        if(video.currentTime >= timestamp + window_size){{
                            video.currentTime = timestamp;
                        }}
                    }},1);
                }}
                
                function pause(){{
                    var video= document.getElementById("window");
                    video.pause();
                }}
            </script>
        </body>
    '''))

# Here you can enter your activities
activities = ["running", "walking", "biking"]

switch = [Button(description=(name +' ' + str(i))) for i, name in enumerate(activities)]

combined = HBox([items for items in switch], layout = Layout(flex_flow='row wrap'))

def upon_clicked(btn):
    print(f'Classified as {btn.description}.')
    for n in range(len(activities)):
        switch[n].style.button_color = 'white'
    btn.style.button_color = 'gray'
    
# while True:
#     if keyboard.is_pressed("a"):
#         print("You pressed 'a'.")
#         break

for n in range(len(activities)):
    switch[n].on_click(upon_clicked)

display_html(timestamp, window_size)
display(combined)

In [2]:
%gui asyncio

In [9]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

from ipywidgets import IntSlider, Output
slider = IntSlider()
out = Output()

async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(switch, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

slider



IntSlider(value=0)

In [32]:
class VideoLabeler():
    def __init__(self, activities: list[str]) -> None:
        self.activities = activities
        self.new_label = ''

    def labeling(self, video_file, timestamp, window_size) -> str:
        # print(f'Current timestamp: {timestamp}')
        self.display_html(video_file, timestamp, window_size)
        return self.display_buttons()

    def display_html(self, video_file, timestamp, window_size) -> None:
        display(HTML(f'''
            <body>
                <video id="window" width="500px" src="{video_file}"></video><br>
                <button onclick="play()">Play</button>
                <button onclick="pause()">Pause</button>
                <script type="text/javascript">
                    var timestamp = {timestamp}
                    var window_size = {window_size}

                    function play(){{
                        var video= document.getElementById("window");
                        if (video.currentTime < timestamp || video.currentTime >= timestamp + window_size){{
                            video.currentTime = timestamp;
                        }}
                        video.play()
                        setInterval(function(){{
                            if(video.currentTime >= timestamp + window_size){{
                                video.currentTime = timestamp;
                            }}
                        }},1);
                        <!--video.playbackRate = 2-->
                    }}
                
                    function pause(){{
                        var video= document.getElementById("window");
                        video.pause();
                    }}
                </script>
            </body>
        '''))

    def display_buttons(self):
        switch = [Button(description=name) for name in activities]
        combined = HBox([items for items in switch], layout = Layout(flex_flow='row wrap'))

        for n in range(len(self.activities)):
            switch[n].on_click(self.upon_clicked)

        display(combined)

    def upon_clicked(self, btn: Button) -> None:
        # clear_output(wait=False)
        # print(f'Current timestamp: {timestamp}')
        # self.display_html(timestamp)
        # self.display_buttons()
        # print(f"Previous classified as '{btn.description}'.")        

        for n in range(len(activities)):
            switch[n].style.button_color = 'white'
        btn.style.button_color = 'gray'
        
        self.new_label = btn.description


vid_lab = VideoLabeler(["running", "walking", "biking"])
print(vid_lab.labeling('Timer2.mp4', 4, 2))

print(vid_lab.new_label)

None

